# CS3244, Machine Learning, Semester 1, 2024/25

### Decision Tree to classify Iris flowers

To run this exercise, you will need to install the ***pydotplus*** package. Use Anaconda Navigator or the command line to install the package.

![Iris image](https://raw.githubusercontent.com/justmarkham/scikit-learn-videos/84f03ae1d048482471f2a9ca85b0c649730cc269/images/03_iris.png "Image from https://github.com/justmarkham/scikit-learn-videos/blob/master/03_getting_started_with_iris.ipynb")
*Image from https://github.com/justmarkham/scikit-learn-videos/blob/master/03_getting_started_with_iris.ipynb*

The Iris dataset contains 50 samples each of three types of the Iris flower: Iris Setosa, Iris Versicolor, and Iris Virginica. For each sample, the following measurements are given: sepal length, sepal width, petal width, petal length.

Run the experiment to build a decision tree for classifying Iris flowers based on their measurements.

From the visualization of the tree, write down a rule that correctly classifies all cases of Iris Setosa. How many disjunction of rules would cover all cases of positive instances of Iris Versicolor? What accuracy would the decision tree be if pruned to depth 2?

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(iris.data, iris.target)

from IPython.display import Image
import pydotplus 

dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

### Decision Tree on Australian Credit Dataset

The **Australian** credit approval dataset illustrates multiple practical issues.
* Missing feature values: This is very common in practice. This can be handled in various ways.
     * Removing instances with missing feature values. This may be fine if the number of such instances is small. However, the issue still has to be handled when the predictor is deployed.
     * Imputing (predicting and filling in) the missing values. May affect performance if prediction is poor.
     * Encoding the missing value as a special value. This may be appropriate if the value is not missing at random and being missing actually provides some information.
     * Algorithm specific method. Decision trees have specific methods for handling missing values (e.g. averaging over both paths down the tree at the missing node) but this is not implemented in Scikit Learn. 
* The Australian dataset has a mix of continuous and categorical feature.
* For confidentiality purposes, feature names and values have been changed into meaningless symbols in the dataset. As a result, we cannot use our exploit knowledge of the problem to construct better predictors. One question is how to handle the categorical feature.
 * If the feature is an ordinal variable, i.e. the values are ordered, it may sometimes be useful to map the values to integers or reals, particularly if we expect the target value to change monotonically with the value of the feature.
 * If the feature is a nominal variable, i.e. the values cannot be ordered, mapping the values to integers or reals may not make sense. Some decision tree algorithms can do a multiway split at the variable with one child for each possible variable value. However, Scikit Learn simply treats all variables as integers/reals and do binary tests with $\leq$.
 
 
In this dataset, missing values are handled by imputation: mean values are used for continuous variables, and mode is used for categorical variables. For datasets like this, one strength of decision trees is that it is somewhat less sensitive to appropriate processing of variables.

Before running the experiments, answer the following in Archipelago.

Do you think the decision tree algorithm is sensitive to scaling of the variables? Yes or No.

In [ ]:
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import neighbors
from sklearn.metrics import accuracy_score

X, y = load_svmlight_file("australian.txt")
# Should really repeat this with many random splits to get more reliable results; try various splits
# by changing random_state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Scale and split data
scaler = preprocessing.MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Decision tree on original data
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("DT accuracy with original data: " + "{0:.2f}".format(accuracy))

# Decision tree on scaled data
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train_scaled, y_train)
predict = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predict)
print("DT accuracy with scaled data: " + "{0:.2f}".format(accuracy) + "\n")

# Nearest neighbour on original data
clf = neighbors.KNeighborsClassifier(1)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("NN accuracy with original data: " + "{0:.2f}".format(accuracy))

# Nearest neighbour on scaled data
clf = neighbors.KNeighborsClassifier(1)
clf = clf.fit(X_train_scaled, y_train)
predict = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predict)
print("NN accuracy with scaled data: " + "{0:.2f}".format(accuracy) + "\n")



### Pruning, Bagging and Boosting Decision Trees on Australian Credit
Continuing on the Australian credit dataset, first try pruning the tree with the complexity parameter $\alpha = 0.1$.  
Then try bagging with 10 trees.  
Finally try boosting with 10 trees.  

Comment on the results.

In [ ]:
# First print out decision tree accuracy on original data
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("DT accuracy with original data: " + "{0:.2f}".format(accuracy))

# Pruned decision tree on original data
clf = tree.DecisionTreeClassifier(random_state=42,ccp_alpha=0.1)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("Pruned decision tree accuracy: " + "{0:.2f}".format(accuracy))

# Bagged decision tree on original data
clf = BaggingClassifier(tree.DecisionTreeClassifier(random_state=42),
                            n_estimators=10,random_state=42)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("Bagged decision tree accuracy: " + "{0:.2f}".format(accuracy))

# Boosted decision tree on original data
clf = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=42),
                         n_estimators=10,random_state=42)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
accuracy = accuracy_score(y_test, predict)
print("Boosted decision tree accuracy: " + "{0:.2f}".format(accuracy))

### Decision Tree, Boosted DT and Bagged DT on DISTINCT

Some problems require large representation when represented as a decision tree.
One such problem is DISTINCT defined as  
$DISTINCT(x,y) = (x_1 \neq y_1) ~OR~ (x_2 \neq y_2) \ldots ~OR~ (x_d \neq y_d )$   
where $x$ and $y$ are $d$-dimensional binary vectors.  

To represent DISTINCT, a decision tree requires at least $2^d$ leaves. Note also, DISTINCT can be represented by a disjunction of $d$ formulas $(x_1 \neq y_1), \ldots, (x_d \neq y_d )$ which is a small representation.

We will try learning DISTINCT using a decision tree, boosted decision trees of depth 2, and bagged decision trees of depth 2. Before running the experiment, predict which methods will do well.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree

train_size = 1000
test_size = 1000
input_size = 20

np.random.seed(0)

# Construct training and test sets
train_data = np.random.randint(2,size=(train_size,input_size))
train_data[:,int(input_size/2):] = train_data[:,:int(input_size/2)] # make equal
# Create labels
train_label = np.zeros(train_size)
for i in range(0,train_size):
    distinct = np.random.randint(2) # Equally likely to be distinct
    if (distinct == 1):
        train_label[i] = 1
        rand_pos = np.random.randint(input_size/2) # randomly select an input and make it different
        train_data[i][rand_pos] = (train_data[i][rand_pos]+1)%2
        
test_data = np.random.randint(2,size=(test_size,input_size))
test_data[:,int(input_size/2):] = test_data[:,:int(input_size/2)] # make equal
test_label = np.zeros(test_size)
for i in range(0,test_size):
    distinct = np.random.randint(2)
    if (distinct == 1):
        test_label[i] = 1
        rand_pos = np.random.randint(input_size/2)
        test_data[i][rand_pos] = (test_data[i][rand_pos]+1)%2

clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(train_data, train_label)
predict = clf.predict(test_data)
accuracy = accuracy_score(test_label, predict)
print("DT accuracy on DISTINCT: " + "{0:.2f}".format(accuracy))

clf = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=2),
                         n_estimators=20,random_state=42)
clf = clf.fit(train_data, train_label)
predict = clf.predict(test_data)
accuracy = accuracy_score(test_label, predict)
print("Boosted decision tree accuracy on DISTINCT: " + "{0:.2f}".format(accuracy))

# Bagged decision tree on original data
clf = BaggingClassifier(tree.DecisionTreeClassifier(max_depth=2),
                            n_estimators=20,random_state=42)
clf = clf.fit(train_data, train_label)
predict = clf.predict(test_data)
accuracy = accuracy_score(test_label, predict)
print("Bagged decision tree accuracy on DISTINCT: " + "{0:.2f}".format(accuracy))